In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# 📂 Chargement des données
data_path = "C:/Users/Admin.local/Documents/Projet_final_data/Piketty_data/elections_socio_fusionnes.csv"
df = pd.read_csv(data_path, dtype={"codecommune": str, "année": int})


In [2]:
df.columns

Index(['codecommune', 'exprimes', 'voteG', 'voteCG', 'voteC', 'voteCD',
       'voteD', 'voteTG', 'voteTD', 'voteGCG', 'voteDCD', 'pvoteG', 'pvoteCG',
       'pvoteC', 'pvoteCD', 'pvoteD', 'pvoteTG', 'pvoteTD', 'pvoteTGratio',
       'pvoteTDratio', 'exprimesT2', 'voteT2_ED', 'voteT2_D', 'voteT2_CD',
       'voteT2_C', 'voteT2_G', 'pvoteT2_ED', 'pvoteT2_D', 'pvoteT2_CD',
       'pvoteT2_C', 'pvoteT2_G', 'pvoteT2_EDratio', 'pvoteT2_Dratio',
       'pvoteT2_CDratio', 'pvoteT2_Cratio', 'pvoteT2_Gratio', 'année',
       'nomcommune', 'cluster_1er_tour', 'cluster_2nd_tour',
       'cluster_corrige_1er_tour', 'revratio', 'pchom', 'pouvr', 'pcadr',
       'pibratio'],
      dtype='object')

In [3]:
df.head()

,codecommune,exprimes,voteG,voteCG,voteC,voteCD,voteD,voteTG,voteTD,voteGCG,...,année,nomcommune,cluster_1er_tour,cluster_2nd_tour,cluster_corrige_1er_tour,revratio,pchom,pouvr,pcadr,pibratio
0,01001,200.0,5.0,47.0,45.0,81.0,22.0,74.5,125.5,52.0,...,1965,ABERGEMENT-CLEMENCIAT,2,0,2,0.942476,0.000000,0.270833,0.000000,0.943578
1,01002,93.0,3.0,33.0,18.0,34.0,5.0,45.0,48.0,36.0,...,1965,ABERGEMENT-DE-VAREY,1,0,1,0.643500,0.000000,0.135135,0.000000,0.644252
2,01004,3827.0,45.0,1453.0,526.0,1522.0,281.0,1761.0,2066.0,1498.0,...,1965,AMBERIEU-EN-BUGEY,1,0,1,0.946163,0.004113,0.440617,0.026735,0.947268
3,01005,322.0,5.0,64.0,80.0,144.0,29.0,109.0,213.0,69.0,...,1965,AMBERIEUX-EN-DOMBES,0,2,0,0.877826,0.000000,0.282486,0.022599,0.878851
4,01006,69.0,0.0,10.0,12.0,42.0,5.0,16.0,53.0,10.0,...,1965,AMBLEON,3,2,3,0.706180,0.000000,0.294118,0.000000,0.707005


In [ ]:

# 📌 Ajout des votes du 1ᵉʳ tour dans les features du 2ᵉ tour
features_2nd_tour = [
    'exprimesT2', 'voteT2_ED', 'voteT2_D', 'voteT2_CD', 'voteT2_C', 'voteT2_G',
    'pvoteT2_ED', 'pvoteT2_D', 'pvoteT2_CD', 'pvoteT2_C', 'pvoteT2_G',
    'pvoteT2_EDratio', 'pvoteT2_Dratio', 'pvoteT2_CDratio', 'pvoteT2_Cratio', 'pvoteT2_Gratio',
    'revratio', 'pchom', 'pouvr', 'pcadr', 'pibratio',
    # Ajout des résultats du 1er tour
    'pvoteG', 'pvoteCG', 'pvoteC', 'pvoteCD', 'pvoteD', 'pvoteTG', 'pvoteTD',
    'pvoteTGratio', 'pvoteTDratio'
]
target_columns = ['pvoteT2_ED', 'pvoteT2_D', 'pvoteT2_CD', 'pvoteT2_C', 'pvoteT2_G']

# 📌 Split Chronologique pour l’Entraînement
train = df[df["année"] <= 2012]
test = df[df["année"] == 2017]
validation = df[df["année"] == 2022]

# 📌 Fonction pour entraîner et évaluer XGBoost
def train_evaluate_xgboost(train, test, features):
    X_train, y_train = train[features], train[target_columns]
    X_test, y_test = test[features], test[target_columns]

    # Définition et entraînement du modèle XGBoost
    model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.1, max_depth=6)
    model.fit(X_train, y_train)

    # Prédictions et évaluation
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred, multioutput='uniform_average')
    
    return model, mae, y_pred

# 📌 Entraînement et Évaluation
print("🔍 Entraînement du modèle pour le 2ᵉ tour avec votes du 1ᵉʳ tour...")
model_2nd_tour, mae_2nd_tour, y_pred_test = train_evaluate_xgboost(train, test, features_2nd_tour)

# 📊 Affichage des résultats
print(f"📉 MAE pour le 2ᵉ tour (Test 2017) : {mae_2nd_tour:.4f}")

# ✅ Validation finale sur 2022
X_validation, y_validation = validation[features_2nd_tour], validation[target_columns]
y_pred_validation = model_2nd_tour.predict(X_validation)

mae_validation = mean_absolute_error(y_validation, y_pred_validation, multioutput='uniform_average')
print(f"📊 Validation Finale (2022) - MAE : {mae_validation:.4f}")


In [10]:


# ✅ Consolidation des résultats sur toute la France
df_results = pd.DataFrame(y_pred_validation, columns=target_columns)
df_results["année"] = validation["année"].values
df_results["exprimesT2"] = validation["exprimesT2"].values

# 📌 Calcul des résultats nationaux pondérés par le nombre de votes exprimés
df_results_national = df_results.copy()
df_results_national["exprimesT2_total"] = df_results_national.groupby("année")["exprimesT2"].transform("sum")

for col in target_columns:
    df_results_national[col] = (df_results_national[col] * df_results_national["exprimesT2"]) / df_results_national["exprimesT2_total"]

# Sélection des résultats nationaux finaux
df_national_summary = df_results_national.groupby("année")[target_columns].sum()

# 📊 Affichage des résultats nationaux
print("\n📊 Résultats consolidés sur la France (normalisés) :")
print(df_national_summary)

# ✅ Sauvegarde des résultats
result_path = "C:/Users/Admin.local/Documents/Projet_final_data/Piketty_data/resultats_nationaux_2nd_tour_normalises.csv"
df_national_summary.to_csv(result_path)
print(f"✅ Fichier des résultats nationaux sauvegardé : {result_path}")



📊 Résultats consolidés sur la France (normalisés) :
       pvoteT2_ED  pvoteT2_D  pvoteT2_CD  pvoteT2_C  pvoteT2_G
année                                                         
2022     0.639802   0.079943   -0.000876   0.708897  -0.000011
✅ Fichier des résultats nationaux sauvegardé : C:/Users/Admin.local/Documents/Projet_final_data/Piketty_data/resultats_nationaux_2nd_tour_normalises.csv


In [11]:
df_national_summary

,pvoteT2_ED,pvoteT2_D,pvoteT2_CD,pvoteT2_C,pvoteT2_G
année,,,,,
2022,0.639802,0.079943,-0.000876,0.708897,-0.000011


In [12]:
# somme du df sur la ligne
total=df_national_summary.sum(axis=1)
print(total)

année
2022    1.427754
dtype: float64


In [13]:
# mise à l'échelle
df_national_summary = df_national_summary.div(total, axis=0)

In [14]:
df_national_summary

,pvoteT2_ED,pvoteT2_D,pvoteT2_CD,pvoteT2_C,pvoteT2_G
année,,,,,
2022,0.448118,0.055992,-0.000614,0.496512,-0.000008


In [15]:
# ✅ Consolidation des résultats sur toute la France
df_results = pd.DataFrame(y_pred_validation, columns=target_columns)
df_results["année"] = validation["année"].values
df_results["exprimesT2"] = validation["exprimesT2"].values

# 📌 Calcul des résultats nationaux pondérés par le nombre de votes exprimés
df_results_national = df_results.copy()
df_results_national["exprimesT2_total"] = df_results_national.groupby("année")["exprimesT2"].transform("sum")

for col in target_columns:
    df_results_national[col] = (df_results_national[col] * df_results_national["exprimesT2"]) / df_results_national["exprimesT2_total"]

# Sélection des résultats nationaux finaux
df_national_summary = df_results_national.groupby("année")[target_columns].sum()

# 📌 Normalisation pour que la somme des pourcentages fasse exactement 100 %
df_national_summary = df_national_summary.div(df_national_summary.sum(axis=1), axis=0) * 100

# 📊 Affichage des résultats nationaux
print("\n📊 Résultats consolidés sur la France (normalisés à 100%) :")
print(df_national_summary)

# ✅ Sauvegarde des résultats
result_path = "C:/Users/Admin.local/Documents/Projet_final_data/Piketty_data/resultats_nationaux_2nd_tour_normalises.csv"
df_national_summary.to_csv(result_path)
print(f"✅ Fichier des résultats nationaux sauvegardé : {result_path}")


📊 Résultats consolidés sur la France (normalisés à 100%) :
       pvoteT2_ED  pvoteT2_D  pvoteT2_CD  pvoteT2_C  pvoteT2_G
année                                                         
2022    44.811784   5.599189   -0.061388  49.651183  -0.000767
✅ Fichier des résultats nationaux sauvegardé : C:/Users/Admin.local/Documents/Projet_final_data/Piketty_data/resultats_nationaux_2nd_tour_normalises.csv
